In [1]:
import os

In [2]:
%pwd

'/home/srak/Documents/pers/end-to-end-chest-cancer/research'

In [3]:
os.chdir("../")
%pwd

'/home/srak/Documents/pers/end-to-end-chest-cancer'

In [30]:
config = read_yaml(CONFIG_FILE_PATH)
config.data_ingestion.unzip_dir

[2024-08-01 12:59:01,737: INFO: common: yaml file: config/config.yaml loaded completed.]


'artifacts/data_ingestion'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    


In [5]:
from ChestCancerClassification.constants import *
from ChestCancerClassification.utils.common import read_yaml, create_directories


In [23]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [24]:
import os
import urllib.request as request
import zipfile
import gdown
from ChestCancerClassification import logger
from ChestCancerClassification.utils.common import get_size

In [32]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig) -> None:
        self.config = config
        
    def download_file(self) -> str:
        try:
            dataset_url = self.config.source_URL
            zip_download_url = self.config.local_data_file
            os.makedirs(self.config.root_dir, exist_ok=True)
            
            logger.info(f"Downloading dataset into {zip_download_url}")
            file_id = dataset_url.split("/")[-2]
            prefix = "https://drive.google.com/uc?/export=download&id="
            gdown.download(prefix+file_id, zip_download_url)
            
            logger.info("Download completed.")
        except Exception as ex:
            raise ex
    
    
    def extract_file(self):
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path, exist_ok=True)
            
            with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
                zip_ref.extractall(unzip_path)
            
            logger.info("File Extraction completed.")
            
        except Exception as ex:
            raise ex

In [33]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingesion = DataIngestion(config=data_ingestion_config)
    data_ingesion.download_file()
    data_ingesion.extract_file()
except Exception as ex:
    raise ex

[2024-08-01 13:03:14,166: INFO: common: yaml file: config/config.yaml loaded completed.]
[2024-08-01 13:03:14,170: INFO: common: yaml file: params.yaml loaded completed.]
[2024-08-01 13:03:14,180: INFO: common: Created dir at path: artifacts]
[2024-08-01 13:03:14,183: INFO: common: Created dir at path: artifacts/data_ingestion]
[2024-08-01 13:03:14,185: INFO: 663402032: Downloading dataset into artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1wHY8slF0EDnpW8rveruK0xI9U7LIPaRv
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1wHY8slF0EDnpW8rveruK0xI9U7LIPaRv&confirm=t&uuid=8025ed97-aafb-476d-9bee-55baca004b63
To: /home/srak/Documents/pers/end-to-end-chest-cancer/artifacts/data_ingestion/data.zip
100%|██████████| 124M/124M [00:20<00:00, 6.10MB/s] 

[2024-08-01 13:03:40,177: INFO: 663402032: Download completed.]


[2024-08-01 13:03:41,472: INFO: 663402032: File Extraction completed.]
